In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[]

In [18]:
import numpy as np
a=np.zeros(9)
a.shape
np.random.randint(9)
np.array(0).astype(np.float64)

array(0.)

In [3]:
import gym.envs.atari

OSError: [WinError 126] A megadott modul nem található

In [2]:
tf.device('/physical_device:GPU:0')

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 183us/sample - loss: 0.2950 - accuracy: 0.9143
Epoch 2/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.1438 - accuracy: 0.9574
Epoch 3/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.1085 - accuracy: 0.9672
Epoch 4/5
60000/60000 [==============================] - 7s 111us/sample - loss: 0.0878 - accuracy: 0.9732
Epoch 5/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.0753 - accuracy: 0.9762


In [4]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 7s 114us/sample - loss: 0.0661 - accuracy: 0.9786
Epoch 2/5
60000/60000 [==============================] - 7s 113us/sample - loss: 0.0566 - accuracy: 0.9823
Epoch 3/5
60000/60000 [==============================] - 7s 110us/sample - loss: 0.0531 - accuracy: 0.9826
Epoch 4/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.0482 - accuracy: 0.9846
Epoch 5/5
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0445 - accuracy: 0.9855
